In [ ]:
import glob
import os
import sys
import random
import time
import numpy as np
import cv2
from tensorflow.keras.models import load_model

try:
    sys.path.append(
        glob.glob("../carla/dist/carla-*%d.%d-%s.egg"
            % (sys.version_info.major,sys.version_info.minor,"win-amd64" if os.name == "nt" else "linux-x86_64",))[0])
except IndexError:
    pass

import carla

CAMERA_POSITION =  [1.3, 0.0, 2.3]
CAMERA_HEIGHT = 480
CAMERA_WIDTH = 960
CAMERA_FOV = 120
CAMERA_LEFT = -60.0
CAMERA_RIGHT = 60.0

COMBINED_IMAGE_SHAPE = (160, 704, 3)

MODEL_INPUT_SIZE = (10,) + COMBINED_IMAGE_SHAPE

MODEL_OUTPUT_SIZE = (5,)


In [ ]:
model = load_model('models/LTC_CNN3D_model-0.9464.hdf5')

In [ ]:
def process_img(image):
    i = np.array(image.raw_data)
    i2 = i.reshape((image.height, image.width, 4))
    i3 = i2[:, :, :3]
    """ cv2.imshow("", i3)
    cv2.waitKey(1) """
    return i3#/255.0

In [ ]:
def camera_rgb_install():
    camera_positions = [
        carla.Transform(carla.Location(x=CAMERA_POSITION[0], z=CAMERA_POSITION[2]), carla.Rotation(yaw=CAMERA_LEFT)),  # Left
        carla.Transform(carla.Location(x=CAMERA_POSITION[0], z=CAMERA_POSITION[2])),  # Center
        carla.Transform(carla.Location(x=CAMERA_POSITION[0], z=CAMERA_POSITION[2]),carla.Rotation(yaw=CAMERA_RIGHT)),
    ]  # Right

    cameras = []
    for camera_transform in camera_positions:
        camera_blueprint = world.get_blueprint_library().find("sensor.camera.rgb")
        camera_blueprint.set_attribute("image_size_x", str(CAMERA_WIDTH))
        camera_blueprint.set_attribute("image_size_y", str(CAMERA_HEIGHT))
        camera_blueprint.set_attribute("fov", str(CAMERA_FOV))
        camera = world.spawn_actor(camera_blueprint, camera_transform, attach_to=ego_vehicle)
        actor_list.append(camera)
        cameras.append(camera)
    return cameras


def combine_images(images):
    combined_image = np.zeros(COMBINED_IMAGE_SHAPE, dtype=np.float32)

    return combined_image


actor_list = []

try:
    client = carla.Client("localhost", 2000)
    client.set_timeout(3.0)

    # Once we have a client we can retrieve the world that is currently running.
    world = client.get_world()

    # Retrieve the spectator object
    spectator = world.get_spectator()

    # Get the location and rotation of the spectator through its transform
    transform = spectator.get_transform()

    location = transform.location
    rotation = transform.rotation

    blueprint_library = world.get_blueprint_library()

    # Set the spectator with an empty transform
    # spectator.set_transform(carla.Transform())

    # This will set the spectator at the origin of the map, with 0 degrees
    # pitch, yaw and roll - a good way to orient yourself in the map

    vehicle_blueprints = blueprint_library.filter("*vehicle*")

    spawn_points = world.get_map().get_spawn_points()

  

    ego_vehicle = world.spawn_actor(
        blueprint_library.filter("etron")[0], random.choice(spawn_points)
    )
    actor_list.append(ego_vehicle)
    car_transform = ego_vehicle.get_transform()
    spectator.set_transform(
        carla.Transform(
            carla.Location(
                x=car_transform.location.x,
                y=car_transform.location.y,
                z=car_transform.location.z + 10,
            ),
            carla.Rotation(
                pitch=-60,
                yaw=car_transform.rotation.yaw,
                roll=car_transform.rotation.roll,
            ),
        )
    )

    """ for i in range(0, 50):
        npc_vehicle = world.try_spawn_actor(
            random.choice(vehicle_blueprints), random.choice(spawn_points)
        )
        if npc_vehicle is not None:
            # npc_vehicle.set_autopilot(True)
            actor_list.append(npc_vehicle) """

    """ postavljanje senzora na auto """
    # kamera RGB
    camera_list = camera_rgb_install()

    ego_vehicle.get_control().steer = 0.0
    ego_vehicle.get_control().throttle = 0.0
    ego_vehicle.get_control().brake = 0.0
    ego_vehicle.get_control().hand_brake = False
    ego_vehicle.get_control().reverse = False

    seconds = 100

    image_queue = []
    camera_images = []
    for frame in range(1):
        for camera in camera_list:
            camera.listen(lambda image: camera_images.append(process_img(image)))

        time.sleep(1)
        combined_image = np.hstack(
            (camera_images[0], camera_images[1], camera_images[2])
        )
        cv2.imshow("", combined_image)
        cv2.waitKey(1)
        image_queue.append(camera_images)
        camera_images.clear()

    """ for _ in range(seconds):
        # Synchronize the CARLA world
        world.tick()

        image_queue = []
        camera_images = []
        for frame in range(10):
            for camera in camera_list:
                camera.listen(lambda image: camera_images.append(process_img(image)))
            combined_image = np.hstack(
                (camera_images[0], camera_images[1], camera_images[2])
            )
            cv2.imshow("", combined_image)
            cv2.waitKey(1)
            image_queue.append(camera_images)

        # Get car controls
        control = ego_vehicle.get_control() """

    time.sleep(5)

finally:
    for actor in actor_list:
        actor.destroy()
    print("done.")